In [23]:
%pip install peft==0.4.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2025-05-06 11:41:15.958618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746531676.378484      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746531676.501115      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
import pandas as pd

num_samples = 100000

df = pd.read_csv("/kaggle/input/recipenlg/RecipeNLG_dataset.csv", nrows = num_samples)
df = df.drop(columns=['Unnamed: 0', 'link', 'source'])
df.head()

,title,ingredients,directions,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [4]:
formatted_texts = []

for index, row in df.iterrows():
    title = row['title']
    ingredients = row['ingredients']
    directions = row['directions']
    ner = row['NER']

    text = (
        f"Recipe Title: {title}\n\n"
        f"Ingredients:\n{ingredients}\n\n"
        f"Instructions:\n{directions}\n\n"
        f"Food Entities: {ner}"
    )

    formatted_texts.append(text)

df['text'] = formatted_texts

In [5]:
df.head()

,title,ingredients,directions,NER,text
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...",Recipe Title: No-Bake Nut Cookies\n\nIngredien...
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom...",Recipe Title: Jewell Ball'S Chicken\n\nIngredi...
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar...","Recipe Title: Creamy Corn\n\nIngredients:\n[""2..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo...",Recipe Title: Chicken Funny\n\nIngredients:\n[...
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu...",Recipe Title: Reeses Cups(Candy) \n\nIngredie...


In [7]:
ids = []
a_masks = []

max_length = 256

for index, row in df.iterrows():
    tokenized_text = tokenizer(
                    row['text'],
                    truncation=True,
                    padding="max_length",
                    max_length=max_length,
                    return_tensors="pt"
                    )

    ids.append(tokenized_text['input_ids'])
    a_masks.append(tokenized_text['attention_mask'])

In [8]:
print(ids[0].shape)
print(a_masks[0].shape)

torch.Size([1, 256])
torch.Size([1, 256])


In [10]:
import torch

final_ids = []
final_a_masks = []

for i in ids:
    input_ids = torch.squeeze(i, dim=0)
    final_ids.append(input_ids.tolist())

for i in a_masks:
    a_mask = torch.squeeze(i, dim=0)
    final_a_masks.append(a_mask.tolist())

In [11]:
df['input_ids'] = final_ids
df['attention_masks'] = final_a_masks

In [12]:
df.head()

,title,ingredients,directions,NER,text,input_ids,attention_masks
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...",Recipe Title: No-Bake Nut Cookies\n\nIngredien...,"[37523, 11851, 25, 1400, 12, 33, 539, 11959, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom...",Recipe Title: Jewell Ball'S Chicken\n\nIngredi...,"[37523, 11851, 25, 3370, 695, 6932, 6, 50, 164...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar...","Recipe Title: Creamy Corn\n\nIngredients:\n[""2...","[37523, 11851, 25, 19979, 88, 11424, 198, 198,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo...",Recipe Title: Chicken Funny\n\nIngredients:\n[...,"[37523, 11851, 25, 16405, 40473, 198, 198, 412...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu...",Recipe Title: Reeses Cups(Candy) \n\nIngredie...,"[37523, 11851, 25, 797, 274, 274, 41611, 7, 34...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [13]:
from datasets import Dataset

df = Dataset.from_pandas(df)

print(df)
print(type(df))  # Should output: <class 'datasets.arrow_dataset.Dataset'>

Dataset({
    features: ['title', 'ingredients', 'directions', 'NER', 'text', 'input_ids', 'attention_masks'],
    num_rows: 100000
})
<class 'datasets.arrow_dataset.Dataset'>


In [14]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size = 0.1, random_state = 42)

In [15]:
import peft
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=2,
    lora_alpha=8,
    target_modules=["c_attn"],
    lora_dropout=0.05, 
    bias="none",
    task_type="CAUSAL_LM"
)

In [20]:
peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 196,608 || all params: 355,019,776 || trainable%: 0.05537945018589612
None


In [21]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 1024)
        (wpe): Embedding(1024, 1024)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPT2Block(
            (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=1024, out_features=3072, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=2, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=2, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
  

In [22]:
from torchinfo import summary

summary(peft_model)

Layer (type:depth-idx)                                            Param #
PeftModelForCausalLM                                              --
├─LoraModel: 1-1                                                  --
│    └─GPT2LMHeadModel: 2-1                                       --
│    │    └─GPT2Model: 3-1                                        355,019,776
│    │    └─Linear: 3-2                                           (51,463,168)
Total params: 406,482,944
Trainable params: 196,608
Non-trainable params: 406,286,336

In [24]:
import transformers
from transformers import TrainingArguments, Trainer
import os

output_directory = os.path.join("../cache/working", "peft_lab_outputs")
training_args = TrainingArguments(
    report_to="none",
    output_dir="./gpt2-lora",
    auto_find_batch_size=True,
    learning_rate= 3e-2, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=100
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=df,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

"./gpt2-lora"

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.133700
200,3.107600
300,3.684800
400,3.179300
500,3.094000
600,3.087900
700,3.029200
800,3.256100
900,3.407700
1000,3.172500


'./gpt2-lora'

In [ ]:
#prompt = "Generate a chocolate chip cookie recipe with these ingredients: flour, sugar, eggs, chocolate chips\n\nRecipe Title:"
#prompt = tokenizer(prompt,  return_tensors="pt")

#params = {
#                'max_length': 400,
#                'num_return_sequences': 1,
#                'temperature': 0.7,
#                'top_k': 50,
#                'top_p': 0.9,
#                'do_sample': True,
#                'no_repeat_ngram_size': 2,
#                'early_stopping': True
#            }

In [ ]:
#outputs = model.generate(
#            **prompt,
#            **params
#        )

#print(outputs)

In [ ]:
#generated_text = tokenizer.decode(
#    outputs[0], 
#    skip_special_tokens=True
#)

#print(generated_text)